# Day 2

Today, we will start using nf-core pipelines to find differentially abundant genes in our dataset. 
We are using data from the following paper: https://www.nature.com/articles/s41593-023-01350-3#Sec10

1. Please take some time to read through the paper and understand their approach, hypotheses and goals.

What was the objective of the study?

- The study's objective was to investigate how patients with chronic neuropathic pain react to opioid administration and withdrawal. Using a mouse model of spared nerve injury (SNI), the authors administered oxycodone and compared gene expression changes with control (sham) and non-opioid groups.

What do the conditions mean?

- oxy: mice that were given oxycodone 


- sal: mice that were given a saline (control group)

What do the genotypes mean?

- SNI: spared nerve injury


- Sham: healthy / control group (which also had surgery, but no chronic pain)

Imagine you are the bioinformatician in the group who conducted this study. They hand you the raw files and ask you to analyze them.

What would you do?    
- The aim would be to perform a differential gene expression analysis. For this, the pipeline we looked at yesterday could be suitable or at least a starting point.
- This approach will hopefully allow to identify transcriptional changes associated with neuropathic pain, opioid exposure and withdrawal

Which groups would you compare to each other?  
- SNI-Oxy vs. Sham-Oxy to find out what differs between SNI / Sham genotypes with drug treatment  
- SNI-Oxy vs. SNI-sal to find out the genetic effects of the drug treatment in SNI mice  
- Sham-Oxy vs. Sham-sal to find out the genetic effects of the drug treatment in Sham mice (according to the paper, research on this has already been done)  
  
Please also mention which outcome you would expect to see from each comparison.  
- 1) a number of differntially expressed genes that are involved in the different reactions to drug treatment and withdrawal in SNI / Sham conditions  
- 2) a number of differentially expressed genes linked to the organism's transcriptional response to oxycodone in the SNI condition    
- 3) a number of differentially expressed genes linked to the organism's transcriptional response to oxycodone without SNI  

Your group gave you a very suboptimal excel sheet (conditions_runs_oxy_project.xlsx) to get the information you need for each run they uploaded to the SRA.<br>
So, instead of directly diving into downloading the data and starting the analysis, you first need to sort the lazy table.<br>
Use Python and Pandas to get the table into a more sensible order.<br>
Then, perform some overview analysis and plot the results
1. How many samples do you have per condition?
2. How many samples do you have per genotype?
3. How often do you have each condition per genotype?

In [ ]:
import pandas as pd

conditions_table = ("conditions_runs_oxy_project.xlsx")

# with "run" as index column: 
df = pd.read_excel(conditions_table, index_col="Run")

# other ideas that were discussed: 
# df.fillna(False) 
# df.replace("x",True)
# or: one condition column with "sal" / "oxy"

# 1. How many samples do you have per condition?
condition_counts = pd.Series({
    "Oxy": (df["Condition: Oxy"] == "x").sum(),
    "Sal": (df["condition: Sal"] == "x").sum()
})


print("Samples per condition:")
print(condition_counts.to_string())

# other idea: 0 and 1, then count


Samples per condition:
Oxy    8
Sal    8


In [26]:

# 2. How many samples do you have per genotype?

genotype_counts = pd.Series({
    "SNI": (df["Genotype: SNI"] == "x").sum(),
    "Sham": (df["Genotype: Sham"] == "x").sum()
})

print("Samples per genotype:")
print(genotype_counts.to_string())

Samples per genotype:
SNI     8
Sham    8


In [27]:
# 3. How often do you have each condition per genotype?
cond_per_geno = pd.crosstab(
    df["Genotype: SNI"] == "x",
    df["Condition: Oxy"] == "x"
)
cond_per_geno.index = ["Sham", "SNI"] 
cond_per_geno.columns = ["Sal", "Oxy"]

print("Condition per genotype:")
print(cond_per_geno)

Condition per genotype:
      Sal  Oxy
Sham    4    4
SNI     4    4


They were so kind to also provide you with the information of the number of bases per run, so that you can know how much space the data will take on your Cluster.<br>
Add a new column to your fancy table with this information (base_counts.csv) and sort your dataframe according to this information and the condition.

Then select the 2 smallest runs from your dataset and download them from SRA (maybe an nf-core pipeline can help here?...)

In [ ]:
base_counts = "base_counts.csv"

bases = pd.read_csv(base_counts, index_col="Run")

# merge based on "Run" column
df = df.merge(bases, on="Run")
df

,Patient,RNA-seq,DNA-seq,condition: Sal,Condition: Oxy,Genotype: SNI,Genotype: Sham,Bases
Run,,,,,,,,
SRR23195505,?,x,NaN,x,NaN,x,NaN,6922564500
SRR23195506,?,x,NaN,NaN,x,NaN,x,7859530800
SRR23195507,?,x,NaN,x,NaN,NaN,x,8063298900
SRR23195508,?,x,NaN,NaN,x,x,NaN,6927786900
SRR23195509,?,x,NaN,NaN,x,x,NaN,7003550100
SRR23195510,?,x,NaN,x,NaN,x,NaN,7377388500
SRR23195511,?,x,NaN,NaN,x,NaN,x,6456390900
SRR23195512,?,x,NaN,x,NaN,NaN,x,7462857900
SRR23195513,?,x,NaN,x,NaN,x,NaN,8099181600


In [31]:
sorted = df.sort_values(by="Bases", ascending=1)
sorted

,Patient,RNA-seq,DNA-seq,condition: Sal,Condition: Oxy,Genotype: SNI,Genotype: Sham,Bases
Run,,,,,,,,
SRR23195516,?,x,NaN,NaN,x,x,NaN,6203117700
SRR23195511,?,x,NaN,NaN,x,NaN,x,6456390900
SRR23195517,?,x,NaN,NaN,x,x,NaN,6863840400
SRR23195505,?,x,NaN,x,NaN,x,NaN,6922564500
SRR23195508,?,x,NaN,NaN,x,x,NaN,6927786900
SRR23195519,?,x,NaN,NaN,x,NaN,x,6996050100
SRR23195509,?,x,NaN,NaN,x,x,NaN,7003550100
SRR23195514,?,x,NaN,NaN,x,NaN,x,7226808600
SRR23195510,?,x,NaN,x,NaN,x,NaN,7377388500


In [ ]:
# fetchngs pipeline

!nextflow run nf-core/fetchngs --input computational-workflows-2025/notebooks/day_02/ids.csv -profile docker --outdir day02/fetchngs_out --max_memory "6GB"


While your files are downloading, get back to the paper and explain how you would try to reproduce the analysis.<br>

The paper did a differential gene analysis on the following pairs:
- SNI-Oxy versus Sham-Sal
- SNI-Sal versus Sham-Sal
- Sham-Oxy versus Sham-Sal
I would try to reproduce the analysis as described in the "Bioninformatics analysis" part of the Methods section.   
The method describes:  
- Read alignment and DGE with HISAT2, HTSeq and DSeq2
- creating plots
- analysis of enriched pathways with IPA

Since this section does not state any detais on how the analysis was performed, which parameters were used etc, and does not link to a github or similar, it could become difficult to reproduce the analysis exactly.


When you are done with this shout, so we can discuss the different ideas.